<a href="https://colab.research.google.com/github/javier1974-hub/Delineador/blob/main/Delineador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from scipy.io import loadmat
import numpy as np
import scipy.signal as sp
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
signal = np.genfromtxt('/content/drive/MyDrive/Colab Notebooks/Delineador/sujeto122_6.csv',dtype= float, delimiter=',')

In [ ]:
plt.plot(signal[0:1500])
plt.ylabel('some numbers')
plt.show()

In [ ]:
h0 = np.array([0.125, 0.375, 0.375, 0.125 ])
g0 = np.array([2, -2])

In [ ]:
h = sp.upfirdn([1], h0, 4)
g = sp.upfirdn([1], g0, 4)
#print(h)
#print(g)

In [ ]:
d1=sp.convolve(signal,g, mode = 'same');
a1=sp.convolve(signal,h, mode = 'same');

h1=sp.upfirdn([1],h,2);
g1=sp.upfirdn([1],g,2);

d2=sp.convolve(a1,g1, mode = 'same');
a2=sp.convolve(a1,h1, mode = 'same');

h2=sp.upfirdn([1],h1,2);
g2=sp.upfirdn([1],g1,2);

d3=sp.convolve(a2,g2, mode = 'same');
a3=sp.convolve(a2,h2, mode = 'same');

h3=sp.upfirdn([1],h2,2);
g3=sp.upfirdn([1],g2,2);

d4=sp.convolve(a3,g3, mode = 'same');
a4=sp.convolve(a3,h3, mode = 'same');

h4=sp.upfirdn([1],h3,2);
g4=sp.upfirdn([1],g3,2);

d5=sp.convolve(a4,g4, mode = 'same');
a5=sp.convolve(a4,h4, mode = 'same');


In [ ]:
#
plt.plot(signal[600:1000])
plt.plot(np.abs(d1[600:1000]))
plt.plot(d1[600:1000])
plt.plot(np.abs(d2[600:1000]))
plt.plot(d2[600:1000])
plt.plot(np.abs(d3[600:1000]))
plt.plot(d3[600:1000])
plt.plot(np.abs(d4[600:1000]))
plt.plot(d4[600:1000])
plt.plot(np.abs(d5[600:1000]))
plt.plot(d5[600:1000])

In [ ]:
plt.plot(signal[200:230])

In [ ]:
def rms(x):
  return np.sqrt(x.dot(x)/x.size)

In [ ]:
def ismax(d,index,thres):
  if ((d[index-1] < d[index]) and (d[index] > d[index+1]) and (d[index]>thres)):
    return True
  else:
    return False

In [ ]:
def ismin(d,index,thres):
  if ((d[index-1] > d[index]) and (d[index] < d[index+1]) and (d[index]< thres)):
    return True
  else:
    return False

In [ ]:
E1= rms(d1)
E2= 2*rms(d2)
E3= 2*rms(d3)
E4= 0.5*rms(d4)
E5= 0.5*rms(d5)

print([E1, E2, E3, E4, E5])

LIMITE = 60            #para PTBDB = 30
LIMITE_QRS = 40         #para PTBDB = 20
cont_MAXIMO_1 = 0
cont_MAXIMO_2 = 0
cont_MINIMO_1 = 0
cont_MINIMO_2 = 0

hay_end = 0
hay_on = 0
hay_R = 0


k = 0
p = 0
nfirst = 0
nlast = 0
npre = 0
npost = 0
R=np.zeros(len(signal))
R1=np.zeros(len(signal))
R2=np.zeros(len(signal))
R3=np.zeros(len(signal))
S=np.zeros(len(signal))
QRSend=np.zeros(len(signal))
Q=np.zeros(len(signal))
QRSon=np.zeros(len(signal))
Rprima=np.zeros(len(signal))
RR=0
pos_S = 0
pos_Q = 0
pos_R = 0
pos_QRSend = 0
pos_QRSon = 0
QRSendQRSend = []
QRSonQRSon = []
pos_QRSendprima = 0
RprimaRprima = 0
posicion_R =0


In [ ]:
def HacerNuevaMuestra(d,index,thres):
  pos_MAX_1 = 0
  pos_MAX_2 = 0
  pos_MAX_3 = 0
  pos_MIN_1 = 0
  pos_MIN_2 = 0
  pos_MIN_3 = 0
  cont_MAXIMO_1 = 0
  cont_MINIMO_1 = 0
  cont_MAXIMO_2 = 0
  cont_MINIMO_2 = 0
  if ismax(d,index,thres):
    pos_MAX_1 = i
    estado_actual = 'MAXIMO_1'
  if ismin(d,index,thres):
    pos_MIN_2 = i
    estado_actual = 'MINIMO_2'

In [ ]:
def HacerMaximo1(d,index,thres):

  cont_MAXIMO_1 = cont_MAXIMO_1 + 1

  if ismin(d,index,thres):
    pos_MIN_1 = index
    estado_actual = 'MINIMO_1'
  if(cont_MAXIMO_1 > 2*LIMITE or (np.sign(d(index-1)) == -1 and np.sign(d(index+1)) == 1)):
    cont_MAXIMO_1 = 0
    pos_MIN_1 = 0
    estado_actual = 'NUEVA_MUESTRA'

In [ ]:
def HacerMinimo1(d,index,thres):

  cont_MINIMO_1 = cont_MINIMO_1 + 1

  if ismax(d,index,thres):
    pos_MAX_3 = i
    estado_actual = 'RS'
  if(cont_MINIMO_1 > 2*LIMITE):
    cont_MINIMO_1 = 0
    pos_MAX_3 = 0
    estado_actual = 'R'

In [ ]:
def HacerMinimo2(d,index,thres):

  cont_MINIMO_2 = cont_MINIMO_2 + 1

  if ismax(d,index,thres):
    pos_MAX_2 = i
    estado_actual = 'MAXIMO_2'
  if(cont_MINIMO_2 > 3*LIMITE  or (np.sign(d(index-1)) == 1 and np.sign(d(index+1)) == -1)):
    cont_MINIMO_2 = 0
    pos_MAX_2 = 0
    estado_actual = 'NUEVA_MUESTRA'

In [ ]:
def HacerMaximo2(d,index,thres):

  cont_MAXIMO_2 = cont_MAXIMO_2 + 1

  if ismin(d,index,thres):
    pos_MIN_3 = i
    estado_actual = 'QR'
  if(cont_MAXIMO_2 > 2*LIMITE):
    cont_MAXIMO_2 = 0
    pos_MIN_3 = 0
    estado_actual = 'R1'

In [ ]:
estado_actual = ['NUEVA_MUESTRA','MAXIMO_1','MINIMO_1','MINIMO_2','MAXIMO_2','R' ,'R1','RS','QR']
estado_anterior = estado_actual

In [ ]:
def DelinearQRS(d,index,thres):
  match estado_actual:
    case 'NUEVA_MUESTRA':
      HacerNuevaMuestra(d,index,thres)
    case 'MAXIMO_1':
      HacerMaximo1(d,index,thres)
    case 'MINIMO_1':
      HacerMinimo1(d,index,thres)
    case 'MINIMO_2':
      HacerMinimo2(d,index,thres)
    case 'MAXIMO_2':
      HacerMaximo2(d,index,thres)
    case 'R':
      pass
    case 'R1':
      pass
    case 'RS':
      pass
    case 'QR':
      pass

    case _:
      pass

In [ ]:
for i in range(2,(len(d3)-1)):
  DelinearQRS(d3,i,E3)

In [ ]:
signal_aux=d3[0:1500]
peaks, _ = sp.find_peaks(signal_aux, height=(0.2, 0.8))
plt.plot(signal_aux)
plt.plot(peaks, signal_aux[peaks], "x")
plt.show()

In [ ]:
signal_aux=d3[0:1500]
peaks, _ = sp.find_peaks(signal_aux, height=0.2)
plt.plot(signal_aux)
plt.plot(peaks, signal_aux[peaks], "x")
plt.show()

In [ ]:
signal_aux=d3[0:1500]
peaks, _ = sp.find_peaks(-signal_aux, height=0.2)
plt.plot(signal_aux)
plt.plot(peaks, signal_aux[peaks], "x")
plt.show()

In [ ]:
signal_aux=d3[600:1000]
peaks, _ = sp.find_peaks(np.abs(signal_aux), height=0.2)
print(peaks[1])
plt.plot(signal_aux)
plt.plot(peaks, signal_aux[peaks], "x")
plt.show()

In [ ]:
signal_aux=d3[600:1000]
peaks, _ = sp.find_peaks(np.abs(signal_aux), height=0.2)
#plt.plot(signal)
min=sp.argrelmin(np.abs(signal_aux[peaks[0]:peaks[1]]))

print([peaks[0], peaks[1], min[0]])
min1 = min[0] + peaks[0]
plt.plot(np.abs(signal_aux))
plt.plot(peaks, signal_aux[peaks], "x")
plt.plot(min1, signal_aux[min1], "o")
plt.show()

In [ ]:
signal_aux=d3[600:1000]
peaks, _ = sp.find_peaks(np.abs(signal_aux), height=E3)
#plt.plot(signal)
min=sp.argrelmin(np.abs(signal_aux[peaks[0]:peaks[1]]))

#print([peaks[0], peaks[1], min[0]])
min1 = min[0] + peaks[0]
#print(min1)
#print(signal[min1])

fig = go.Figure()
t=np.linspace(1,400, 400)

#fig.add_trace(go.Scatter(x=t, y= signal[600:1000], mode='lines',name='lines'))
fig.add_trace(go.Scatter(x=t, y= d3[600:1000], mode='lines',name='lines'))
fig.add_trace(go.Scatter(x=min1, y=signal[min1+600], mode='markers',name='markers'))

fig.show()


In [ ]:
fig = px.line(signal)
fig.show()

In [ ]:
signal_aux=d3
peaks, _ = sp.find_peaks(np.abs(signal_aux), height=E3, distance=10)

In [ ]:
# para tenerlos de a pares
if (peaks[1]-peaks[0] > 50):
 peaks = np.delete(peaks, 0)

In [ ]:
# para tenerlos de a pares
if (peaks[-1]-peaks[-2] > 50):
 peaks = np.delete(peaks, -1)

In [ ]:
print(peaks)

In [ ]:
fig = go.Figure()
t=np.linspace(1,len(d3), len(d3))

fig.add_trace(go.Scatter(x=t, y= signal, mode='lines',name='lines'))
fig.add_trace(go.Scatter(x=t, y= np.abs(d3), mode='lines',name='lines'))
fig.add_trace(go.Scatter(x=t, y= np.abs(d2), mode='lines',name='lines'))
fig.add_trace(go.Scatter(x=peaks, y= np.abs(d3[peaks]), mode='markers',name='markers'))
#fig.add_trace(go.Scatter(x=min1, y=signal[min1+600], mode='markers',name='markers'))

fig.show()

In [ ]:
posR=np.zeros(int(len(peaks)/2),dtype=int)    # lo tengo que hacer entero porque lo uso como indice para graficar. No puedo usar floats
j=0
for i in range(0,(len(peaks)-1),2):
  R, _ = sp.find_peaks(-np.abs(d3[peaks[i]:peaks[i+1]]))
  posR[j]=R[0]+peaks[i]
  print([peaks[i], R, posR[j]])
  j = j+1

In [ ]:
print(R)
print(len(peaks))
print(posR)
type(posR)

In [ ]:
fig = go.Figure()
t=np.linspace(1,len(d3), len(d3))

fig.add_trace(go.Scatter(x=t, y= signal, mode='lines',name='lines'))
#fig.add_trace(go.Scatter(x=t, y= np.abs(d3), mode='lines',name='lines'))
#fig.add_trace(go.Scatter(x=t, y= np.abs(d2), mode='lines',name='lines'))
#fig.add_trace(go.Scatter(x=posR, y= np.abs(d3[posR]), mode='markers',name='markers'))
fig.add_trace(go.Scatter(x=posR, y=signal[posR], mode='markers',name='markers'))
fig.show()

In [ ]:
ventana = np.ones(150)/150

In [ ]:
print(ventana)

In [ ]:
x = sp.convolve(np.abs(d2),ventana,mode='same')

In [ ]:
fig = go.Figure()
t=np.linspace(1,len(d2), len(d2))

fig.add_trace(go.Scatter(x=t, y= x, mode='lines',name='lines'))
fig.add_trace(go.Scatter(x=t, y= np.abs(d2), mode='lines',name='lines'))
#fig.add_trace(go.Scatter(x=t, y= np.abs(d2), mode='lines',name='lines'))
#fig.add_trace(go.Scatter(x=peaks, y= np.abs(d3[peaks]), mode='markers',name='markers'))
#fig.add_trace(go.Scatter(x=min1, y=signal[min1+600], mode='markers',name='markers'))

fig.show()

In [ ]:
thr = 2*rms(x)
print(thr)

In [ ]:
posQRS, _ = sp.find_peaks(np.abs(x), height=thr, width = 150, distance=300)

In [ ]:
print(len(posQRS))
print(posQRS)

In [ ]:
fig = go.Figure()
t=np.linspace(1,len(d2), len(d2))

fig.add_trace(go.Scatter(x=t, y= x, mode='lines',name='lines'))
#fig.add_trace(go.Scatter(x=t, y= np.abs(d2), mode='lines',name='lines'))
#fig.add_trace(go.Scatter(x=peaks, y= np.abs(d3[peaks]), mode='markers',name='markers'))
fig.add_trace(go.Scatter(x=posQRS, y=x[posQRS], mode='markers',name='markers'))

fig.show()

In [ ]:
candidatosR=np.zeros((len(posQRS),300))
print(candidatosR)
print(candidatosR.shape)

#for i in range(len(posQRS)):
#  print(d3[(posQRS[i]-150):(posQRS[i]+150)])

for i in range(len(posQRS)):
  candidatosR[i] = d3[(posQRS[i]-150):(posQRS[i]+150)]


In [ ]:
print(candidatosR)

In [ ]:
fig = go.Figure()
t=np.linspace(1,len(candidatosR[1,:]), len(candidatosR[1,:]))
print(len(candidatosR[1,:]))
print(len(posQRS))
for i in range(len(posQRS)-1):
  fig.add_trace(go.Scatter(x=t, y= candidatosR[i,:], mode='lines',name='lines'))

fig.show()

In [ ]:
nroQRS = 0
th1 = 0.5*rms(candidatosR[1,:])
th2 = rms(candidatosR[1,:])
th3 = 1.5*rms(candidatosR[1,:])
th4 = 2*rms(candidatosR[1,:])
print(th2)
peaksR, _ = sp.find_peaks(np.abs(candidatosR[nroQRS,:]), height=th2)
print(peaksR)

In [ ]:


fig = go.Figure()
t=np.linspace(1,len(candidatosR[nroQRS,:]), len(candidatosR[nroQRS,:]))

fig.add_trace(go.Scatter(x=t, y= candidatosR[nroQRS,:], mode='lines',name='d3'))
fig.add_trace(go.Scatter(x=t, y= np.abs(d2[(posQRS[nroQRS]-150):(posQRS[nroQRS]+150)]), mode='lines',name='d2'))
fig.add_trace(go.Scatter(x=t, y= np.abs(candidatosR[nroQRS,:]), mode='lines',name='abs(d3)'))
fig.add_hline(y=th1, line_width=3, line_dash="dash", line_color="green")
fig.add_hline(y=th2, line_width=3, line_dash="dash", line_color="red")
fig.add_hline(y=th3, line_width=3, line_dash="dash", line_color="blue")
fig.add_hline(y=th4, line_width=3, line_dash="dash", line_color="black")
fig.add_hline(y=-th1, line_width=3, line_dash="dash", line_color="green")
fig.add_hline(y=-th2, line_width=3, line_dash="dash", line_color="red")
fig.add_hline(y=-th3, line_width=3, line_dash="dash", line_color="blue")
fig.add_hline(y=-th4, line_width=3, line_dash="dash", line_color="black")
fig.add_trace(go.Scatter(x=peaksR, y=candidatosR[nroQRS,:][peaksR], mode='markers',name='markers'))
fig.add_trace(go.Scatter(x=t, y= signal[(posQRS[nroQRS]-150):(posQRS[nroQRS]+150)], mode='lines',name='signal'))

fig.show()



In [ ]:
x = np.array([2, 1, 2, 3, 2, 0, 1, 0])
sp.argrelmin(x,order=1)

In [ ]:
x = np.array([1, 4, 3, 2, 2, 3, 4, 5, 1])
sp.argrelmin(x,order=2)

Tendria que hacer una funcion que parado sobre un extremo (maximo o minimo) se desplace hacia los costados (derecha o izq ) buscando el extremo opuesto. Si empiezo en un maximo, busco minimos. Si estoy en un
minimo, busco un maximo.

le trendria que pasar el punto de inicio, la direccion de busqueda, la ventana de busqueda. Devuelve la posicion del primer punto que cumple.

Podria hacer todo lo anterior solo para uibicar las R.
Luego podria moverme +/- x muestras en torno a la R para clasificar la morfologia del QRS. una vez que tengo la morfolofia. Sé cuantos maximos y minimos buscar en la transformada.
La idea es análoga a las demas ondas ?

In [ ]:
def findMax(scale, point, winSize, dir):
  if dir == 1:
    aux = sp.argrelmax(scale[(point):(point + winSize)], order = 1)
    return  aux[0] + point
  if dir == -1:
    aux = sp.argrelmax(scale[(point - winSize):(point)], order = 1)
    print(aux)
    return  ((point - winSize) + aux[0] )


In [ ]:
def findMin(scale, point, winSize, dir):
  if dir == 1:
    aux = sp.argrelmin(scale[(point):(point + winSize)], order = 1)
    return aux[0] + point
  if dir == -1:
    aux = sp.argrelmin(scale[(point - winSize):(point)], order = 1)
    print(aux)
    return  ((point - winSize) + aux[0] )

In [ ]:
max = findMax(candidatosR[0,:],160, 30, 1)
print(max)
max1 = findMax(candidatosR[0,:],160, 30, -1)
print(max1)

In [ ]:
aux = sp.argrelmax(candidatosR[0,:][(160 - 30):(160)], order = 1)
print(aux)

In [ ]:
print(posQRS[0])
print(d3[(posQRS[0]):(posQRS[0] + 20)])

sp.argrelmax(np.abs(d3[(posQRS[0]):(posQRS[0] + 20)]), order = 2)

In [ ]:
fig = go.Figure()

t=np.linspace(1,300, 300)
fig.add_trace(go.Scatter(x=t, y= np.abs(candidatosR[0,:]), mode='lines',name='d3'))


fig.show()